In [1]:
import leafmap

In [11]:
country = "TheGambia"
head = 2

In [12]:
leafmap.download_ms_buildings(
    country, 
    out_dir=f"buildings {country}", 
    merge_output=f"{country}_ms.shp", 
    head=head
    )

Found 6 links for TheGambia
Reading 1 of 2: buildings/33303200.geojson
Reading 2 of 2: buildings/33303201.geojson
Merging GeoDataFrames ...
Saving merged file to TheGambia_ms.shp ...
Saved merged file to TheGambia_ms.shp


['buildings/33303200.geojson', 'buildings/33303201.geojson']

In [ ]:
m = leafmap.Map()
m.add_basemap("SATELLITE")
m.add_vector(f"{country}_ms.shp", layer_name="MS Buildings")
m

In [4]:
def quadkey_to_coordinates(quadkey):
    bounding_box = {
        'min_lat': -85.05112878,
        'max_lat': 85.05112878,
        'min_lon': -180.0,
        'max_lon': 180.0
    }

    level = len(quadkey)

    for i in range(level):
        digit = int(quadkey[i])
        mid_lat = (bounding_box['min_lat'] + bounding_box['max_lat']) / 2
        mid_lon = (bounding_box['min_lon'] + bounding_box['max_lon']) / 2

        if digit == 0:
            bounding_box['max_lat'] = mid_lat
            bounding_box['max_lon'] = mid_lon
        elif digit == 1:
            bounding_box['max_lat'] = mid_lat
            bounding_box['min_lon'] = mid_lon
        elif digit == 2:
            bounding_box['min_lat'] = mid_lat
            bounding_box['max_lon'] = mid_lon
        elif digit == 3:
            bounding_box['min_lat'] = mid_lat
            bounding_box['min_lon'] = mid_lon

    return {
        'latitude': (bounding_box['min_lat'] + bounding_box['max_lat']) / 2,
        'longitude': (bounding_box['min_lon'] + bounding_box['max_lon']) / 2
    }

# Example usage:
quadkey = "033133031213"
coordinates = quadkey_to_coordinates(quadkey)
print('lat-long: ', coordinates['latitude'], coordinates['longitude'] )

lat-long:  -12.3963681351709 -3.2080078125


In [6]:
def coordinates_to_quadkey(latitude, longitude, zoom):
    n = 2 ** zoom
    x_tile = int((longitude + 180.0) / 360.0 * n)
    y_tile = int((1.0 - (latitude + 90.0) / 180.0) * n)

    quadkey = ""
    for i in range(zoom, 0, -1):
        digit = 0
        mask = 1 << (i - 1)
        if (x_tile & mask) != 0:
            digit += 1
        if (y_tile & mask) != 0:
            digit += 2
        quadkey += str(digit)

    return quadkey

# Example usage:
latitude = 13.47815
longitude = -16.68430
zoom_level = 12
quadkey = coordinates_to_quadkey(latitude, longitude, zoom_level)
print(quadkey)

033123002212


In [7]:
import math

def get_quadkey(lat, lng, zoom):
    x = int((lng + 180) / 360 * (1 << zoom))
    y = int((1 - math.log(math.tan(math.radians(lat)) + 1 / math.cos(math.radians(lat))) / math.pi) / 2 * (1 << zoom))
    quadkey = ''
    for i in range(zoom, 0, -1):
        digit = 0
        mask = 1 << (i - 1)
        if (x & mask) != 0:
            digit += 1
        if (y & mask) != 0:
            digit += 2
        quadkey += str(digit)
    return quadkey

In [15]:
import geopandas as gpd
df = gpd.read_file("buildings/33303200.geojson")

In [20]:
import folium
import matplotlib.pyplot as plt
import mapclassify

In [10]:
from collections import namedtuple
def quadkey_to_tile(qk):
    """Get the tile corresponding to a quadkey

    Parameters
    ----------
    qk : str
        A quadkey string.

    Returns
    -------
    Tile

    """
    Tile = namedtuple("Tile", ["x", "y", "z"])
    if len(qk) == 0:
        return Tile(0, 0, 0)
    xtile, ytile = 0, 0
    for i, digit in enumerate(reversed(qk)):
        mask = 1 << i
        if digit == "1":
            xtile = xtile | mask
        elif digit == "2":
            ytile = ytile | mask
        elif digit == "3":
            xtile = xtile | mask
            ytile = ytile | mask
        elif digit != "0":
            warnings.warn(
                "QuadKeyError will not derive from ValueError in mercantile 2.0.",
                DeprecationWarning,
            )
            raise QuadKeyError("Unexpected quadkey digit: %r", digit)
    return Tile(xtile, ytile, i + 1)

In [11]:
quadkey_to_tile('033303200')

Tile(x=232, y=236, z=9)

In [9]:
get_quadkey(13.47815, -16.68430, 9)

'033303200'

In [ ]:
def coord_to_quadkey():
    